# Learning to Rank Demo | Activate 2020

## Overview

In this notebook, we will train a Learning to Rank model from user click data using ml4ir.

#### Key Takeaways
- How to install and get started with ml4ir as a script and a library
- Defining a ranking pipeline from scratch
- Transfer learning for ml4ir models
- Serving models trained on ml4ir

#### Learning to Rank
The goal of Learning to Rank(LTR) is to come up with a ranking function to generate an optimal ordering of a list of documents. 

## Contents

1. [Install ml4ir](#Install-ml4ir)
1. [Look at the Data](#Look-at-the-Data)
1. [Define the FeatureConfig](#Define-the-FeatureConfig)
1. [Define the ModelConfig](#Define-the-ModelConfig)
1. [Using ml4ir as a script](#Using-ml4ir-as-a-script)
1. [Using ml4ir as a library](#Using-ml4ir-as-a-library)
    1. [Setup](#Setup)
    1. [Load the FeatureConfig](#Load-the-FeatureConfig)
    1. [Create a RelevanceDataset](#Create-a-RelevanceDataset)
    1. [Define an InteractionModel](#Define-an-InteractionModel)
    1. [Define losses, metrics and optimizer](#Define-losses,-metrics-and-optimizer)
    1. [Define the scoring function, or the Scorer](#Define-a-scoring-function,-or-the-Scorer)
    1. [Combine it all to create a RankingModel](#Combine-it-all-to-create-a-RankingModel)
    1. [Train and Evaluate your RankingModel](#Train-and-Evaluate-your-RankingModel)
    1. [Save the trained RankingModel](#Save-the-trained-RankingModel)
1. [Let's try some Transfer Learning](#Let's-try-some-Transfer-Learning)
    1. [What does ml4ir save?](#What-does-ml4ir-save?)
    1. [Using pre-trained character embeddings](#Using-pre-trained-character-embeddings)
1. [Model Serving](#Model-Serving)
    1. [JVM Serving Logic](#JVM-Serving-Logic)
    1. [Serving your Ranking Model](#Serving-your-Ranking-Model)

## Install ml4ir

In [1]:
!pip install ml4ir

Looking in indexes: https://pypi.python.org/simple


  Using cached tensorboard-2.0.1-py3-none-any.whl (3.8 MB)


  Using cached grpcio_gcp-0.2.2-py2.py3-none-any.whl (9.4 kB)
  Using cached google_cloud_vision-0.42.0-py2.py3-none-any.whl (435 kB)
  Using cached google_cloud_spanner-1.13.0-py2.py3-none-any.whl (212 kB)
Processing /Users/ashish.srinivasa/Library/Caches/pip/wheels/19/b5/2f/1cc3cf2b31e7a9cd1508731212526d9550271274d351c96f16/google_apitools-0.5.31-py3-none-any.whl
  Using cached google_cloud_language-1.3.0-py2.py3-none-any.whl (83 kB)
  Using cached google_cloud_dlp-0.13.0-py2.py3-none-any.whl (151 kB)
  Using cached google_cloud_videointelligence-1.13.0-py2.py3-none-any.whl (177 kB)
ERROR: tensorflow-transform 0.15.0 has requirement absl-py<0.9,>=0.7, but you'll have absl-py 0.9.0 which is incompatible.
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.0.2
    Uninstalling tensorboard-2.0.2:
      Successfully uninstalled tensorboard-2.0.2
  Attempting uninstall: google-apitools
    Found existing installation: google-apitools 0.5.28
    Uninstalling 

## Look at the data

In [9]:
import pandas as pd

df_train = pd.read_csv("../ml4ir/applications/ranking/tests/data/csv/train/file_0.csv")
df_train.head(7)

,query_id,rank,query_text,domain_name,text_match_score,page_views_score,quality_score,name_match,domain_id,clicked
0,query_0,1,UQHA3QP4ZVO,domain_0,1.101297,0.002044,0.00000,0,0,1
1,query_0,2,UQHA3QP4ZVO,domain_0,0.380570,0.004078,0.30103,0,0,0
2,query_1,1,8M3NWYX4E6I,domain_1,1.024334,0.008686,0.30103,1,1,1
3,query_1,2,8M3NWYX4E6I,domain_1,0.821515,0.200264,0.00000,1,1,0
4,query_1,3,8M3NWYX4E6I,domain_1,0.821323,0.200264,0.00000,0,1,0
5,query_1,4,8M3NWYX4E6I,domain_1,0.821515,0.200264,0.00000,0,1,0
6,query_1,5,8M3NWYX4E6I,domain_1,0.821323,0.200264,0.00000,1,1,0


## Define the FeatureConfig

| Feature          | Type    | TFRecord Type | Usage                                    |
| ---------------- | -------- | ------------- | ---------------------------------------- |
| query_text       | Text     | Context       | Character Embeddings -> biLSTM Encoding  |
| domain_name      | Text     | Context       | VocabLookup -> Categorical Embedding     |
| text_match_score | Numeric  | Sequence      | float                                    |
| page_views_score | Numeric  | Sequence      | float                                    |
| quality_score    | Numeric  | Sequence      | float                                    |

## Define the ModelConfig

In [10]:
print(open("configs/activate_2020/model_config.yaml").read())

architecture_key: dnn
layers:
  - type: dense
    name: first_dense
    units: 256
    activation: relu
  - type: dropout
    name: first_dropout
    rate: 0.3
  - type: dense
    name: second_dense
    units: 64
    activation: relu
  - type: dense
    name: final_dense
    units: 1
    activation: null



## Using ml4ir as a script

In [1]:
!python ../ml4ir/applications/ranking/pipeline.py \
--data_dir ../ml4ir/applications/ranking/tests/data/csv \
--feature_config configs/activate_2020/feature_config.yaml \
--model_config configs/activate_2020/model_config.yaml \
--data_format csv \
--execution_mode train_inference_evaluate \
--loss_key rank_one_listnet \
--num_epochs 3 \
--models_dir ../models/activate_2020 \
--logs_dir ../logs/activate_2020 \
--run_id activate_demo

INFO: 2020-09-11 00:39:38.109 
Logging initialized. Saving logs to : ../logs/activate_2020/activate_demo
INFO: 2020-09-11 00:39:38.109 
Run ID: activate_demo
DEBUG: 2020-09-11 00:39:38.109 
CLI args: 
{
    "data_dir": "../ml4ir/applications/ranking/tests/data/csv",
    "data_format": "csv",
    "tfrecord_type": "sequence_example",
    "feature_config": "configs/activate_2020/feature_config.yaml",
    "model_file": "",
    "model_config": "configs/activate_2020/model_config.yaml",
    "optimizer_key": "adam",
    "loss_key": "rank_one_listnet",
    "metrics_keys": "['MRR', 'ACR']",
    "monitor_metric": "new_MRR",
    "monitor_mode": "max",
    "num_epochs": 3,
    "batch_size": 128,
    "learning_rate": 0.01,
    "learning_rate_decay": 0.9,
    "learning_rate_decay_steps": 1000,
    "compute_intermediate_stats": true,
    "execution_mode": "train_inference_evaluate",
    "random_state": 123,
    "run_id": "activate_demo",
    "run_group": "general",
    "run_notes": "",
    "models_di

INFO: 2020-09-11 00:39:38.161 
Writing SequenceExample protobufs to : ../ml4ir/applications/ranking/tests/data/csv/tfrecord/train/file_0.tfrecord
INFO: 2020-09-11 00:39:39.365 
1 files found under ../ml4ir/applications/ranking/tests/data/csv/tfrecord/train
INFO: 2020-09-11 00:39:42.427 
Created TFRecordDataset from SequenceExample protobufs from 1 files : ['../ml4ir/applications/ranking/tests/data/csv/tfr
INFO: 2020-09-11 00:39:42.428 
1 files found under ../ml4ir/applications/ranking/tests/data/csv/validation
INFO: 2020-09-11 00:39:42.428 
Reading 1 files from [../ml4ir/applications/ranking/tests/data/csv/validation/file_0.csv, ..
INFO: 2020-09-11 00:39:42.428 
Loading dataframe from path : ../ml4ir/applications/ranking/tests/data/csv/validation/file_0.csv
INFO: 2020-09-11 00:39:42.441 
Writing SequenceExample protobufs to : ../ml4ir/applications/ranking/tests/data/csv/tfrecord/validation/file_0.tfrecord
INFO: 2020-09-11 00:39:44.611 
1 files found under ../ml4ir/applications/ranking/

INFO: 2020-09-11 00:39:53.127 
Training Model
INFO: 2020-09-11 00:39:53.131 
Starting Epoch : 1
INFO: 2020-09-11 00:39:53.131 
{}
Epoch 1/3
INFO: 2020-09-11 00:39:59.983 
[epoch: 1 | batch: 0] {'batch': 0, 'size': 128, 'loss': 2.066517, 'old_MRR': 0.8084635, 'new_MRR': 0.5960504, 'old_ACR': 1.5859375, 'new_ACR': 2.4140625}
     11/Unknown - 9s 838ms/step - loss: 1.8755 - old_MRR: 0.7875 - new_MRR: 0.6796 - old_ACR: 1.6491 - new_ACR: 2.0547INFO: 2020-09-11 00:40:02.662 
Evaluating Model
INFO: 2020-09-11 00:40:05.722 
Completed evaluating model
INFO: 2020-09-11 00:40:05.722 
None

Epoch 00001: val_new_MRR improved from -inf to 0.71098, saving model to ../models/activate_2020/activate_demo/checkpoint.tf
INFO: 2020-09-11 00:40:34.885 
End of Epoch 1
INFO: 2020-09-11 00:40:34.886 
{'loss': 1.87550827589902, 'old_MRR': 0.7874729, 'new_MRR': 0.6796089, 'old_ACR': 1.6491477, 'new_ACR': 2.0546875, 'val_loss': 1.8289068395441228, 'val_old_MRR': 0.7827933, 'val_new_MRR': 0.7109778, 'val_old_ACR':

## Using ml4ir as a library

<img src="images/model_framework.png" alt="ml4ir Architecture" style="width: 500px;" align="center"/>

### Setup

In [2]:
MODEL_CONFIG_PATH = "configs/activate_2020/model_config.yaml"
FEATURE_CONFIG_PATH = "configs/activate_2020/feature_config.yaml"

DATA_DIR = "../ml4ir/applications/ranking/tests/data/csv"
MODELS_DIR = '../models/activate_2020'
LOGS_DIR = '../logs/activate_2020'

MAX_SEQUENCE_SIZE = 25

In [3]:
import logging
import tensorflow as tf
import os

from tensorflow.keras.metrics import Metric
from tensorflow.keras.optimizers import Optimizer
from typing import List, Union, Type

from ml4ir.base.io.local_io import LocalIO
from ml4ir.base.io.file_io import FileIO
from ml4ir.base.config.keys import *
from ml4ir.base.data.relevance_dataset import RelevanceDataset
from ml4ir.base.features.feature_config import FeatureConfig, SequenceExampleFeatureConfig
from ml4ir.base.model.scoring.scoring_model import RelevanceScorer
from ml4ir.base.model.relevance_model import RelevanceModel
from ml4ir.base.model.scoring.interaction_model import InteractionModel, UnivariateInteractionModel
from ml4ir.base.model.losses.loss_base import RelevanceLossBase
from ml4ir.base.model.optimizer import get_optimizer
from ml4ir.applications.ranking.model.ranking_model import RankingModel
from ml4ir.applications.ranking.config.keys import LossKey, MetricKey, ScoringTypeKey
from ml4ir.applications.ranking.model.losses import loss_factory
from ml4ir.applications.ranking.model.metrics import metric_factory

In [18]:
# Set up file I/O handler
file_io : FileIO = LocalIO()
    
# Create directories for models and logs
file_io.make_directory(LOGS_DIR, clear_dir=True)
file_io.make_directory(MODELS_DIR, clear_dir=True)

# Set up logger
logger = logging.getLogger()

tf.get_logger().setLevel("ERROR")
tf.autograph.set_verbosity(3)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

### Load the FeatureConfig

In [7]:
feature_config: SequenceExampleFeatureConfig = FeatureConfig.get_instance(
    tfrecord_type=TFRecordTypeKey.SEQUENCE_EXAMPLE,
    feature_config_dict=file_io.read_yaml(FEATURE_CONFIG_PATH),
    logger=logger)
print("Training features\n-----------------")
print("\n".join(feature_config.get_train_features(key="name")))

Training features
-----------------
text_match_score
page_views_score
quality_score
query_text
domain_name


### Create a RelevanceDataset

In [9]:
def get_relevance_dataset(preprocessing_keys_to_fns={}):

    return RelevanceDataset(data_dir=DATA_DIR,
                            data_format=DataFormatKey.CSV,
                            feature_config=feature_config,
                            tfrecord_type=TFRecordTypeKey.SEQUENCE_EXAMPLE,
                            max_sequence_size=MAX_SEQUENCE_SIZE,
                            batch_size=128,
                            preprocessing_keys_to_fns=preprocessing_keys_to_fns,
                            file_io=file_io,
                            logger=logger)

ranking_dataset = get_relevance_dataset()

### Define an InteractionModel

In [10]:
interaction_model: InteractionModel = UnivariateInteractionModel(
                                            feature_config=feature_config,
                                            tfrecord_type=TFRecordTypeKey.SEQUENCE_EXAMPLE,
                                            max_sequence_size=MAX_SEQUENCE_SIZE,
                                            file_io=file_io,
                                        )

### Define losses, metrics and optimizer
##### Use predefined losses, metrics and optimizers or create your own!

In [29]:
# Define loss object from loss key
loss: RelevanceLossBase = loss_factory.get_loss(
                                loss_key=LossKey.RANK_ONE_LISTNET,
                                scoring_type=ScoringTypeKey.POINTWISE)
    
# Define metrics objects from metrics keys
metric_keys = ["MRR", "ACR"]
metrics: List[Union[Type[Metric], str]] = [metric_factory.get_metric(metric_key=m) for m in metric_keys]
    
# Define optimizer
optimizer: Optimizer = get_optimizer(
                            optimizer_key=OptimizerKey.ADAM,
                            learning_rate=0.001
                        )

### Define a scoring function, or the Scorer

In [12]:
scorer: RelevanceScorer = RelevanceScorer.from_model_config_file(
    model_config_file=MODEL_CONFIG_PATH,
    interaction_model=interaction_model,
    loss=loss,
    logger=logger,
    file_io=file_io,
)

### Combine it all to create a RankingModel

In [31]:
logger.setLevel(logging.DEBUG)

ranking_model: RelevanceModel = RankingModel(
                                    feature_config=feature_config,
                                    tfrecord_type=TFRecordTypeKey.SEQUENCE_EXAMPLE,
                                    scorer=scorer,
                                    metrics=metrics,
                                    optimizer=optimizer,
                                    file_io=file_io,
                                    logger=logger,
                                )

DEBUG:root:Logger is initialized...
INFO:root:Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
query_text (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
mask (InputLayer)               [(None, None)]       0                                            
__________________________________________________________________________________________________
tf_op_layer_DecodePaddedRaw_1 ( [(None, 1, 20)]      0           query_text[0][0]                 
__________________________________________________________________________________________________
domain_name (InputLayer)        [(None, 1)]          0                                            
______________________________________________

### Train and Evaluate your RankingModel

In [14]:
ranking_model.fit(dataset=ranking_dataset,
                  num_epochs=3, 
                  models_dir=MODELS_DIR,
                  logs_dir=LOGS_DIR,
                  monitor_metric="new_MRR",
                  monitor_mode="max")

Epoch 1/3
      1/Unknown - 6s 6s/step - loss: 2.0716 - old_MRR: 0.8085 - new_MRR: 0.5434WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (0.194856). Check your callbacks.
     11/Unknown - 7s 672ms/step - loss: 1.9772 - old_MRR: 0.7875 - new_MRR: 0.6415
Epoch 00001: val_new_MRR improved from -inf to 0.70102, saving model to ../models/activate_2020/checkpoint.tf
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ../models/activate_2020/checkpoint.tf/assets
11/11 [==============================] - 38s 3s/step - loss: 1.9772 - old_MRR: 0.7875 - new_MRR: 0.6415 - val_loss: 0.0000e+00 - val_old_MRR: 0.0000e+00 - val_new_MRR: 0.0000e+00
Epoch 2/3
10/11 [==========================>...] - ETA: 0s - loss: 1.9204 - old_MRR: 0.7829 - new_MRR: 0.6761
Epoch 00002: val_new_MRR improved from 0.70102 to 0.70455, saving model to ../models/activate_2020/checkpoint.tf
INFO:tensorflow:Assets written to: ..

In [19]:
ranking_model.predict(test_dataset=ranking_dataset.test).sample(10)

,query_id,clicked,name_match,query_text,domain_name,rank,score,new_rank
21,b'query_228',0,1.0,b'bzxq2',b'domain_3',1,0.393578,1
452,b'query_442',1,0.0,b'qx88u',b'domain_2',1,0.213232,4
132,b'query_1264',0,0.0,b'3vpqpjm8kkq8nrb',b'domain_4',3,0.179684,3
170,b'query_608',0,0.0,b'eq6hcyaf85w',b'domain_3',3,0.164251,5
284,b'query_1058',0,1.0,b'mue0jlw2',b'domain_3',3,0.055739,5
187,b'query_385',0,1.0,b'1d07crpq43h',b'domain_0',3,0.152300,3
260,b'query_1053',1,0.0,b'kvff3b',b'domain_3',1,0.696607,1
300,b'query_181',0,0.0,b'y6eeu0x9kh',b'domain_1',4,0.234538,2
414,b'query_109',0,1.0,b'3izevxuus60b',b'domain_4',13,0.032995,13
93,b'query_1135',1,1.0,b'lkg2bqlhmvc',b'domain_0',1,0.723946,1


In [28]:
import pandas as pd

pd.DataFrame(ranking_model.evaluate(test_dataset=ranking_dataset.test)[0], columns=["value"])

,value
query_count,1408.000000
old_ACR,1.650568
new_ACR,1.968750
old_MRR,0.785881
new_MRR,0.697386
perc_improv_ACR,-19.277108
perc_improv_MRR,-11.260586


### Save the trained RankingModel

In [30]:
ranking_model.save(models_dir=MODELS_DIR,
                   preprocessing_keys_to_fns={},
                   required_fields_only=True)

--------

## Let's try some Transfer Learning

### What does ml4ir save?

<img src="images/ml4ir_savedmodel.png" alt="ml4ir Architecture" style="width: 350px;" align="left"/>

### Using pre-trained character embeddings

In [57]:
initialize_layers_dict = {
    "query_text_bytes_embedding" : "models/activate_2020/bytes_embedding.npy"
}
freeze_layers_list = ["query_text_bytes_embedding"]
ranking_model: RelevanceModel = RankingModel(
                                    feature_config=feature_config,
                                    tfrecord_type=TFRecordTypeKey.SEQUENCE_EXAMPLE,
                                    scorer=scorer,
                                    metrics=metrics,
                                    optimizer=optimizer,
                                    initialize_layers_dict=initialize_layers_dict,
                                    freeze_layers_list=freeze_layers_list,
                                    file_io=file_io,
                                    logger=logger,
                                )

INFO:root:Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
query_text (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
mask (InputLayer)               [(None, None)]       0                                            
__________________________________________________________________________________________________
tf_op_layer_DecodePaddedRaw_4 ( [(None, 1, 20)]      0           query_text[0][0]                 
__________________________________________________________________________________________________
domain_name (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________

INFO:root:Setting query_text_bytes_embedding weights from models/activate_2020/bytes_embedding.npy
INFO:root:Freezing query_text_bytes_embedding layer


In [63]:
ranking_model_embeddings = ranking_model.model.get_layer("query_text_bytes_embedding").get_weights()
query_classification_embeddings = np.load("models/activate_2020/bytes_embedding.npy")

assert tf.reduce_any(tf.equal(ranking_model_embeddings, query_classification_embeddings))

print(ranking_model_embeddings)

[array([[-0.03738469,  0.00727513, -0.02006867, ...,  0.01078511,
        -0.028496  , -0.04102874],
       [ 0.00512887,  0.0062821 , -0.0010671 , ...,  0.04945388,
        -0.0132054 , -0.01177131],
       [-0.00937623, -0.03438247,  0.00176773, ...,  0.046078  ,
        -0.0310035 , -0.04288797],
       ...,
       [-0.04410168,  0.0383402 ,  0.03348425, ...,  0.02123589,
         0.02240864, -0.04049417],
       [ 0.03601265,  0.04585798,  0.00272902, ..., -0.00353998,
        -0.04783431,  0.02852656],
       [ 0.04903785, -0.03518286,  0.00195389, ...,  0.03783921,
        -0.01398294,  0.0107099 ]], dtype=float32)]


## Model Serving

### JVM Serving Logic


```
def runQueriesAgainstDocs(
        csvDataPath: String,
        modelPath: String,
        featureConfigPath: String,
        inputTFNode: String,
        scoresTFNode: String): Iterable[(StringMapQueryContextAndDocs, SequenceExample, Array[Float])] = {
  
  val featureConfig = ModelFeaturesConfig.load(featureConfigPath)
  val sequenceExampleBuilder = StringMapSequenceExampleBuilder.withFeatureProcessors(featureConfig)
  val rankingModelConfig = ModelExecutorConfig(inputTFNode, scoresTFNode)
  val rankingModel = new SavedModelBundleExecutor(modelPath, rankingModelConfig)

  val queryContextsAndDocs = StringMapCSVLoader.loadDataFromCSV(csvDataPath, featureConfig)

  queryContextsAndDocs.map {
    case q @ StringMapQueryContextAndDocs(queryContext, docs) =>
      val sequenceExample = sequenceExampleBuilder.build(queryContext, docs)
      (q, sequenceExample, rankingModel(sequenceExample))
  }
}

val allScores: Iterable[
  (StringMapQueryContextAndDocs, SequenceExample, Array[Float])] = runQueriesAgainstDocs(
    pathFor("test_data.csv"),
    pathFor("ranking_model_bundle"),
    pathFor("feature_config.yaml"),
    "serving_tfrecord_protos",
    "ranking_score"
  )
```

### Serving your RankingModel

In [64]:
!cd ../../jvm; mvn test -Dtest=ml4ir.inference.tensorflow.TensorFlowInferenceTest

[INFO] Scanning for projects...
[WARNING] 
[WARNING] Some problems were encountered while building the effective model for ml4ir:ml4ir-inference:jar:0.0.2-SNAPSHOT
[WARNING] 'build.plugins.plugin.version' for org.apache.maven.plugins:maven-compiler-plugin is missing. @ line 87, column 15
[WARNING] 'build.plugins.plugin.version' for org.apache.maven.plugins:maven-jar-plugin is missing. @ line 94, column 15
[WARNING] 
[WARNING] It is highly recommended to fix these problems because they threaten the stability of your build.
[WARNING] 
[WARNING] For this reason, future Maven versions might no longer support building such malformed projects.
[WARNING] 
[INFO] ------------------------------------------------------------------------
[INFO] Reactor Build Order:
[INFO] 
[INFO] ml4ir-parent                                                       [pom]
[INFO] ml4ir-inference                                                    [jar]
[INFO] 
[INFO] -------------------------< ml4ir:ml4ir-parent >-----


Results :

Tests run: 1, Failures: 0, Errors: 0, Skipped: 0

[INFO] ------------------------------------------------------------------------
[INFO] Reactor Summary for ml4ir-parent 0.0.2-SNAPSHOT:
[INFO] 
[INFO] ml4ir-parent ....................................... SUCCESS [  0.002 s]
[INFO] ml4ir-inference .................................... SUCCESS [ 33.024 s]
[INFO] ------------------------------------------------------------------------
[INFO] BUILD SUCCESS
[INFO] ------------------------------------------------------------------------
[INFO] Total time:  33.128 s
[INFO] Finished at: 2020-09-11T01:26:24-07:00
[INFO] ------------------------------------------------------------------------
